In [258]:
import sys, random, math
from collections import Counter
import numpy as np

np.random.seed(1)

In [259]:
f = open("../original/reviews.txt")
raw_reviews = f.readlines()
f.close()

print(len(raw_reviews))

25000


In [260]:
raw_reviews = raw_reviews[0:10000]

In [261]:
tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
print(len(tokens))
print(len(tokens[1]))

10000
127


In [262]:
wrdcnt = Counter()
for review in tokens:
    for word in review:
        wrdcnt[word] -=1
vocab = list(set(map(lambda x:x[0], wrdcnt.most_common() ))) #another way of getting getting the vocab, is set required ? 
print(len(vocab))
vocab[0:5]

49260


['', 'farmhouses', 'cheever', 'fini', 'magnetism']

In [263]:
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i
    
list(word2index)[0:5]

['', 'farmhouses', 'cheever', 'fini', 'magnetism']

In [264]:
concatenated = list()
input_dataset = list()

for review in tokens:
    review_index = list()
    for word in review:
        try:
            review_index.append(word2index[word])#get the number
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(review_index)

concatenated = np.array(concatenated) #one big long list of values

In [265]:
print(len(input_dataset))
print(concatenated.shape)

10000
(2946251,)


In [266]:
#random.shuffle(input_dataset)

# The model 

In [267]:
print(np.random.rand(1, 5)) #different ways of the same thing
print(np.random.random((1,5)))

[[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01]]
[[0.09233859 0.18626021 0.34556073 0.39676747 0.53881673]]


In [268]:
alpha, iterations = (0.05, 2)
hidden_size, window, negative = (50, 2, 5)

w01 = (np.random.rand(len(vocab), hidden_size) - 0.5) * 0.2 #(74075, 50)
w12 = np.random.rand(len(vocab), hidden_size)*0 #(74075, 50)

layer_2_target = np.zeros(negative +1 ) #(1,6) #create random vector 
layer_2_target[0] = 1 #make the first element 1 

In [269]:
print(w01.shape)
print(w12.shape)

(49260, 50)
(49260, 50)


In [270]:
def similar(target='strip'):
  target_index = word2index[target]
  scores = Counter()
    
  for word,index in word2index.items():
    raw_difference = w01[index] - (w01[target_index])
    squared_difference = raw_difference * raw_difference
    scores[word] = -math.sqrt(sum(squared_difference))
  return scores.most_common(10)

In [271]:
sigmoid = lambda x: 1/(1 + np.exp(-x))

In [272]:
for rev_i, review in enumerate(input_dataset * iterations): #just doubles input dataset? 
    for target_i in range(len(review)):
        
        sample_concatenated5 = list(concatenated[(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])
        target_samples  = [review[target_i]] + sample_concatenated5 #6 index returned, first index is correct
        
        left_context = review[max(0,target_i-window):target_i] #two words before
        right_context = review[target_i+1:min(len(review), target_i+1+window)] #two words after 
        
        layer_1 = np.mean(w01[left_context+right_context], axis = 0) #average the columns for 4 rows (50,)
        layer_2 = sigmoid(layer_1.dot(w12[target_samples].T))  #(6,50).T -> (1,6)
        
        layer_2_delta = layer_2 - layer_2_target #only the first one has a 1 (1,6)
        layer_1_delta = layer_2_delta.dot(w12[target_samples]) #(1,6).dot(6,50) = (1,50) spread the error back to the six, should push
        #random 5 towards zero. 
        
        w12[target_samples] -= np.outer(layer_2_delta, layer_1) * alpha #(1,6), (50) - (6,50)
        w01[left_context+right_context] -= layer_1_delta * alpha #substract off the each row of interest, 4 surrounding

    if(rev_i %  50 == 0):
        sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)*iterations)) + "   " + str(similar('beautiful')))
    
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)*iterations)))
        


Progress:0.99995  [('beautiful', -0.0), ('gorgeous', -3.744634814491358), ('tedious', -4.252121730293498), ('magnificent', -4.261944084540728), ('shallow', -4.294753251487127), ('charismatic', -4.306679849652027), ('brutal', -4.307912783130663), ('marvelous', -4.325921599183216), ('creepy', -4.402085856723191), ('fascinating', -4.4054757312232375)])]])]377156124)]

In [257]:
similar("terrible")

[('terrible', -0.0),
 ('superb', -1.9382935721169392),
 ('entertaining', -1.9752831234779957),
 ('decent', -1.9928674892047238),
 ('predictable', -2.0306268811176857),
 ('weak', -2.0330169379006287),
 ('fairly', -2.0415966832232044),
 ('handled', -2.048035298912921),
 ('compelling', -2.1025811386491924),
 ('forgettable', -2.114956665104193)]